In [3]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
np.random.seed(42)
%matplotlib inline

In [ ]:
pd.options.display.max_columns = 999

In [ ]:
plt.rcParams['figure.figsize'] = (10,6)

## Week 12: Predictive Modeling Part 2
Nov 21, 2019


## Housekeeping

- Class is on Tuesday 11/26 next week (same time) because of Thanksgiving
- HW #7 (required) is due next week
    - Includes final project proposal

## Last time

- An introduction to supervised learning and regression with scikit learn
- **Example:** modeling housing prices in Philadelphia
- **Key concepts:**
    - Linear regression
    - Ridge regression with regularization 
    - Test/train split and $k$-fold cross validation
    - Feature engineering
        - Scaling input features
        - Adding polynomial features
        - One-hot encoding + categorical variables
    - Decision trees and random forests

## Today: Predictive modeling continued

**Focus**: featuring engineering and adding spatial based features

- **Part 1:** Revisiting housing prices modeling
- **Part 2:** Predicting bikeshare demand in Philadelphia

## Part 1: Adding spatial features to the housing price model

Recall:

- Modeling residential sale prices for sales in 2018 with data from the Office of Property Assessment
- Model included: property characteristics and ZIP code categorical variables
- Random forest model showed good improvement over standard linear regression (ordinary least squares)

**Let's do some additional feature engineering to see if we can improve the model's accuracy.**

First, let's setup all of the imports we'll need from scikit learn:

In [ ]:
# Preprocessing/Setup
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

# Models
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.ensemble import RandomForestRegressor

# Model selection
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

## Let's load the same data as last week

Query the city's CARTO cloud database to pull the data for all residential sales in 2018:

In [ ]:
import carto2gpd

In [ ]:
# the CARTO API url
carto_url = "https://phl.carto.com/api/v2/sql"

# The table name
table_name = "opa_properties_public"

# Only pull 2018 sales for single family residential properties
where = "sale_date >= '2018-01-01' and sale_date < '2019-01-01' and category_code_description = 'Single Family'"

# Run the query
salesRaw = carto2gpd.get(carto_url, table_name, where=where)

In [ ]:
salesRaw.head()

In [ ]:
len(salesRaw)

## Clean the raw data

- Remove features with a large number of missing values
- Format the ZIP code column
- Trim to sales between \\$3,000 and \\$1 million

In [ ]:
# The feature columns we want to use
cols = [
    "sale_price",
    "total_livable_area",
    "total_area",
    "garage_spaces",
    "fireplaces",
    "number_of_bathrooms",
    "number_of_bedrooms",
    "number_stories",
    "exterior_condition",
    "zip_code",
    "geometry"
]

# Trim to these columns and remove NaNs
sales = salesRaw[cols].dropna()

# Trim zip code to only the first five digits
sales['zip_code'] = sales['zip_code'].astype(str).str.slice(0, 5)

In [ ]:
# Trim very low and very high sales
valid = (sales['sale_price'] > 3000) & (sales['sale_price'] < 1e6)
sales = sales.loc[valid]

In [ ]:
len(sales)

## Remember: One-hot encoding in scikit learn

- The [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) object is a preprocessor that will perform the vectorization step
- The [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) object will help us apply different transformers to numerical and categorical columns

In [ ]:
# Numerical columns
num_cols = [
    "total_livable_area",
    "total_area",
    "garage_spaces",
    "fireplaces",
    "number_of_bathrooms",
    "number_of_bedrooms",
    "number_stories",
]

# Categorical columns
cat_cols = ["exterior_condition", "zip_code"]

In [ ]:
# Set up the column transformer with two transformers
# Scale the numerical columns and one-hot 
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

In [ ]:
# Initialize the pipeline
# NOTE: only use 20 estimators here so it will run in a reasonable time
regr = make_pipeline(
    preprocessor, RandomForestRegressor(n_estimators=20, random_state=42)
)

## Now, let's fit the model (same as last week)

- Use a 70%/30% train/test split — 30% of data is reserved for final testing
- Use the log of the sale price as the target values to regress

In [ ]:
# Split the data 70/30
train_set, test_set = train_test_split(sales, test_size=0.3, random_state=42)

# the target labels
y_train = np.log(train_set["sale_price"])
y_test = np.log(test_set["sale_price"])

In [ ]:
# Fit the training set
regr.fit(train_set, y_train);

In [ ]:
# What's the test score?
regr.score(test_set, y_test)

## Let's plot the top 30 importances

To do so, remember, we'll need to get the names of the columns created during the one-hot encoding process...

In [ ]:
# The one-hot step
ohe = preprocessor.named_transformers_['cat']

# One column for each category type!
ohe_cols = ohe.get_feature_names()

# Full list of columns is numerical + one-hot
features = num_cols + list(ohe_cols)

In [ ]:
import hvplot.pandas

In [ ]:
regressor = regr["randomforestregressor"]

# Create the dataframe with importances
importance = pd.DataFrame(
    {"Feature": features, "Importance": regressor.feature_importances_}
)

# Sort by importance and get the top 30
# MAKE SURE TO SORT IN DESCENDING ORDER!!
importance = importance.sort_values("Importance", ascending=False).iloc[:30]

# Plot
importance.hvplot.barh(
    x="Feature", y="Importance", height=500, flip_yaxis=True
)

## Improving the model even further

- Adding in ZIP code information captures a lot of the neighborhood-based amenity/disamenity properties
- Can we explicitly add new features that also try to capture some of those features?

**Yes, let's add distance-based features**

## Spatial amenity/disamenity features

**The strategy**

- Get the data for a certain type of amenity, e.g., restaurants, bars, or disamenity, e.g., crimes
    - Data sources: 311 requests, crime incidents, Open Street Map
- Use scikit learn's nearest neighbor algorithm to calculate the distance from each sale to its nearest neighbor in the amenity/disamenity datasets

## Examples of new possible features...

Distance from each sale to:

- Universities
- Parks
- City Hall
- Subway Stops
- New Construction Permits
- Aggravated Assaults
- Graffiti 311 Calls
- Abandoned Vehicle 311 Calls

## Example #1: 311 Graffiti Calls

Source: https://www.opendataphilly.org/dataset/311-service-and-information-requests

### Step 1: Download the data from the CARTO database


We'll only pull data from 2018.

In [ ]:
# the 311 table
table_name = "public_cases_fc"

# Peak at the first row of data
carto2gpd.get(carto_url, table_name, limit=1)

In [ ]:
# Select only those for grafitti and in 2018
where_2018 = "requested_datetime >= '01-01-2018' and requested_datetime < '01-01-2019'"
where_grafitti = "service_name = 'Graffiti Removal'"
where = f"{where_2018} and {where_grafitti}"

# Pull the subset we want
graffiti = carto2gpd.get(carto_url, table_name, where=where)

In [ ]:
# Remove rows with missing geometries
graffiti = graffiti.loc[graffiti.geometry.notnull()]

In [ ]:
len(graffiti)

In [ ]:
graffiti.head()

### Step 2: Get the x/y coordinates of both datasets

We will need to:

- We'll want distances in meters (rather than degrees), so we'll convert the CRS to EPSG=3857
- Extract out the x/y coordinates of the geometry column of each dataset (sales and grafitti calls)

In [ ]:
# Do the CRS conversion
sales_3857 = sales.to_crs(epsg=3857)
graffiti_3857 = graffiti.to_crs(epsg=3857)

In [ ]:
def get_xy_from_geometry(df):
    """
    Return a numpy array with two columns, where the 
    first holds the `x` geometry coordinate and the second 
    column holds the `y` geometry coordinate
    """
    x = df.geometry.x
    y = df.geometry.y
    
    return np.column_stack((x, y)) # stack as columns

In [ ]:
# Extract x/y for sales
salesXY = get_xy_from_geometry(sales_3857)

# Extract x/y for grafitti calls
graffitiXY = get_xy_from_geometry(graffiti_3857)

In [ ]:
salesXY.shape

In [ ]:
graffitiXY.shape

### Step 3: Calculate the nearest neighbor distances

For this, we will use the $k$ nearest neighbors algorithm from scikit learn.

For each sale:
- Find the $k$ nearest neighbors in the second dataset (graffiti calls, crimes, etc)
- Calculate the average distance from the sale to those $k$ neighbors

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
# STEP 1: Initialize the algorithm
k = 5
nbrs = NearestNeighbors(n_neighbors=k)

# STEP 2: Fit the algorithm on the "neighbors" dataset
nbrs.fit(graffitiXY)

# STEP 3: Get distances for sale to 
grafDists, grafIndices = nbrs.kneighbors(salesXY)

***Note:** I am using `k=5` here without any real justification. In practice, you would want to try a few different k values to try to identify the best value to use.

## What did we just calculate?

- `grafDists`: For each sale, the distances to the 5 nearest graffiti calls
    - This should have 5 columns and the same length as the sales dataset
- `grafIndices`: For each sale, the index of each of the neighbors in the original dataset
    - This allows you to access the original 311 graffiti data

In [ ]:
print("length of sales = ", len(salesXY))
print("shape of grafDists = ", grafDists.shape)
print("shape of grafIndices = ", grafIndices.shape)

In [ ]:
# The distances from the first sale to the 5 nearest neighbors
grafDists[0]

## Can we reproduce these distances?

In [ ]:
# The coordinates for the first sale
x0, y0 = salesXY[0]
x0, y0

In [ ]:
# The indices for the 5 nearest graffiti calls
grafIndices[0]

In [ ]:
# the graffiti neighbors
sale0_neighbors = graffitiXY[grafIndices[0]]
sale0_neighbors

In [ ]:
# Access the first and second column for x/y values
neighbors_x = sale0_neighbors[:,0]
neighbors_y = sale0_neighbors[:,1]

# The x/y differences between neighbors and first sale coordinates
dx = (neighbors_x - x0)
dy = (neighbors_y - y0)

# The Euclidean dist
manual_dists = (dx**2 + dy**2) ** 0.5

In [ ]:
manual_dists

In [ ]:
grafDists[0]

## Use the log of the average distance as the new feature

We'll average over the column axis (`axis=1`):

In [ ]:
# Average distance to neighbors
avgGrafDist = grafDists.mean(axis=1)

# Set zero distances to be small, but nonzero
# IMPORTANT: THIS WILL AVOID INF DISTANCES WHEN DOING THE LOG
avgGrafDist[avgGrafDist==0] = 1e-5

# Calculate log of distances
sales['logDistGraffiti'] = np.log10(avgGrafDist)

In [ ]:
sales.head()

## Let's plot a hex map of the new feature!

In [ ]:
# Load the City Limits to plot too
import esri2gpd

# From OpenDataPhilly's page
url = "https://services.arcgis.com/fLeGjb7u4uXqeF9q/arcgis/rest/services/City_Limits/FeatureServer/0"
city_limits = esri2gpd.get(url).to_crs(epsg=3857)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10), facecolor=plt.get_cmap('viridis')(0))

# Plot the log of the Graffiti distance
x = salesXY[:,0]
y = salesXY[:,1]
ax.hexbin(x, y, C=sales['logDistGraffiti'].values, gridsize=60)

# Plot the city limits
city_limits.plot(ax=ax, facecolor='none', edgecolor='white', linewidth=4)

ax.set_axis_off()
ax.set_aspect("equal")

## Example #2: Subway stops

We'll use a new package `osm2gpd` to pull subway stops from Open Street Map.

In [ ]:
import osm2gpd

Similar syntax to the other packages to query CARTO and ESRI Map Servers (carto2gpd and esri2gpd)

In [ ]:
osm2gpd.get?

What we need:

1. Bounding box in lat/lng to search within — we can use the city limits
1. The type of feature to search for.
    - For a full list, see: http://wiki.openstreetmap.org/wiki/Map_Features

In [ ]:
# Bounding box from city limits
# CONVERT TO EPSG=4326 FIRST!
lng_min, lat_min, lng_max, lat_max = city_limits.to_crs(epsg=4326).total_bounds

In [ ]:
# Grab subway stations
where="station=subway"

In [ ]:
subway = osm2gpd.get(lng_min, lat_min, lng_max, lat_max, where=where).to_crs(epsg=3857)
subway.head()

## Important: we need to trim to just Philadelphia

- Since we only supplied a rectangular bounding box, it's possible some features will be outside Philadelphia's city limits.
- We can do a spatial join with the city limits to find those featueres within the city bounds.

In [ ]:
subway = gpd.sjoin(subway, city_limits, op="within")

## What data did we get?

The point locations of the Broad St. and Markford-Frankford subway stops!

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

# Plot the subway locations
subway.plot(ax=ax, markersize=5, color='crimson')

# City limits, too
city_limits.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=4)

ax.set_axis_off()

## Now, get the distances to the nearest subway stop

We'll use $k=1$ to get the distance to the nearest stop.

In [ ]:
# STEP 1: x/y coordinates of subway stops (in EPGS=3857)
subwayXY = get_xy_from_geometry(subway.to_crs(epsg=3857))

# STEP 2: Initialize the algorithm
nbrs = NearestNeighbors(n_neighbors=1)

# STEP 3: Fit the algorithm on the "neighbors" dataset
nbrs.fit(subwayXY)

# STEP 4: Get distances for sale to neighbors
subwayDists, subwayIndices = nbrs.kneighbors(salesXY)

# STEP 5: add back to the original dataset
sales['logDistSubway'] = np.log10(subwayDists[:,0])

## Let's plot a hex map again!

In [ ]:
fig, ax = plt.subplots(figsize=(10,10), facecolor=plt.get_cmap('viridis')(0))

# Plot the log of the subway distance
x = salesXY[:,0]
y = salesXY[:,1]
ax.hexbin(x, y, C=np.log10(subwayDists.mean(axis=1)), gridsize=60)

# Plot the city limits
city_limits.plot(ax=ax, facecolor='none', edgecolor='white', linewidth=4)

ax.set_axis_off()
ax.set_aspect("equal")

## Looks like it worked!

# Now, let's re-run our model...did it help?

In [ ]:
# Numerical columns
num_cols = [
    "total_livable_area",
    "total_area",
    "garage_spaces",
    "fireplaces",
    "number_of_bathrooms",
    "number_of_bedrooms",
    "number_stories",
    "logDistGraffiti",
    "logDistSubway"
]

# Categorical columns
cat_cols = ["exterior_condition", "zip_code"]

In [ ]:
# Set up the column transformer with two transformers
# Scale the numerical columns and one-hot 
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

In [ ]:
# Initialize the pipeline
# NOTE: only use 20 estimators here so it will run in a reasonable time
regr = make_pipeline(
    preprocessor, RandomForestRegressor(n_estimators=20, random_state=42)
)

In [ ]:
# Split the data 70/30
train_set, test_set = train_test_split(sales, test_size=0.3, random_state=42)

# the target labels
y_train = np.log(train_set["sale_price"])
y_test = np.log(test_set["sale_price"])

In [ ]:
# Fit the training set
regr.fit(train_set, y_train);

In [ ]:
# What's the test score?
regr.score(test_set, y_test)

## Small improvement!

## How about the top 30 feature importances now?

In [ ]:
def plot_feature_importances(regr, num_cols, preprocessor, top=20, **kwargs):
    """
    Utility function to plot the feature importances from the input
    random forest regressor
    """
    # The one-hot step
    ohe = preprocessor.named_transformers_["cat"]

    # One column for each category type!
    ohe_cols = ohe.get_feature_names()

    # Full list of columns is numerical + one-hot
    features = num_cols + list(ohe_cols)

    # The regressor
    regressor = regr["randomforestregressor"]

    # Create the dataframe with importances
    importance = pd.DataFrame(
        {"Feature": features, "Importance": regressor.feature_importances_}
    )

    # Sort importance in descending order and get the top
    importance = importance.sort_values("Importance", ascending=False).iloc[:top]

    # Plot
    return importance.hvplot.barh(x="Feature", y="Importance", flip_yaxis=True, **kwargs)

In [ ]:
plot_feature_importances(regr, num_cols, preprocessor, top=30, height=500)

## Both new spatial features are in the top 5 in terms of importance!

## Exercise: How about other spatial features?

- I've listed out several other types of potential sources of new distance-based features from OpenDataPhilly
- Choose a few and add new features
- Re-fit the model and evalute the performance on the test set and feature importances

### Universities

New feature: Distance to the *nearest* university/college

- Source: [OpenDataPhilly](https://www.opendataphilly.org/dataset/philadelphia-universities-and-colleges)
- GeoService URL: https://services.arcgis.com/fLeGjb7u4uXqeF9q/ArcGIS/rest/services/Universities_Colleges/FeatureServer/0


### Parks

New feature: Distance to the *nearest* park centroid

* Source: [OpenDataPhilly](https://www.opendataphilly.org/dataset/parks-and-recreation-assets)
* GeoService URL: https://services.arcgis.com/fLeGjb7u4uXqeF9q/ArcGIS/rest/services/PPR_Assets/FeatureServer/0

**Notes** 
- The park geometries are *polygons*, so you'll need to get the `x` and `y` coordinates of the park *centroids* and calculate the distance to these centroids. 
- You can use the `geometry.centroid.x` and `geometry.centroid.y` values to access these coordinates.

### City Hall

New feature: Distance to City Hall.

* Source: [OpenDataPhilly](https://www.opendataphilly.org/dataset/city-landmarks)
* GeoService URL: https://services.arcgis.com/fLeGjb7u4uXqeF9q/ArcGIS/rest/services/CITY_LANDMARKS/FeatureServer/0

**Notes**

- To identify City Hall, you'll need to pull data where "NAME='City Hall'" and "FEAT_TYPE='Municipal Building'"
- As with the parks, the geometry will be a *polygon*, so you should calculate the distance to the *centroid* of the City Hall polygon

### New Construction Permits

New feature: Distance to the 5 nearest new construction permits from 2018

* Source: [OpenDataPhilly](https://www.opendataphilly.org/dataset/licenses-and-inspections-building-permits)
* CARTO table name: "li_permits"

**Notes**

* You can pull new construction permits only by selecting where `permitdescription` equals 'NEW CONSTRUCTION PERMIT'
* You can select permits from only 2018 using the `permitissuedate` column

### Aggravated Assaults

New feature: Distance to the 5 nearest aggravated assaults in 2018

* Source: [OpenDataPhilly](https://www.opendataphilly.org/dataset/crime-incidents)
* CARTO table name: "incidents_part1_part2"

**Notes**

* You can pull aggravated assaults only by selecting where `Text_General_Code` equals 'Aggravated Assault No Firearm' or 'Aggravated Assault Firearm'
* You can select crimes from only 2018 using the `dispatch_date` column

### Abandonded Vehicle 311 Calls

New feature: Distance to the 5 nearest abandoned vehicle 311 calls in 2018

* Source: [OpenDataPhilly](https://www.opendataphilly.org/dataset/311-service-and-information-requests)
* CARTO table name: "public_cases_fc"

**Notes**

* You can pull abandonded vehicle calls only by selecting where `service_name` equals 'Abandoned Vehicle'
* You can select crimes from only 2018 using the `requested_datetime` column

## Part 2: Predicting bikeshare demand in Philadelphia

**The technical problem**: predict bikeshare trip counts for the Indego bikeshare in Philadelphia



## The policy question: how to best expand a bikeshare program?

- Bikeshares typically require substantial capital when first launching, about \\$4,000 to \\$5,000 per bike
- Once launched, revenue from riders typically covers about 80 percent of operating costs
- Ridership peaks in busy downtown hubs, but how to best expand outwards, often to low-density and low-income communities?
- Important cost/benefit questions of how/where to expand to maximize ridership and access and minimize the need for outside subsidies

For more info, see [this blog post](https://www.pewtrusts.org/en/research-and-analysis/blogs/stateline/2016/03/24/despite-popularity-bike-share-programs-often-need-subsidies) from Pew

## Using predictive modeling as a policy tool

- Construct a predictive model for trip counts by stations in a bikeshare
- Use this model to estimate and map out the ridership for potential stations in new areas
- Use the cost per ride to estimate the additional revenue added
- Compare this additional revenue to the cost of adding new stations

## What are the key assumptions here?

**Most important:** adding new stations in new areas will not affect the demand for existing stations.

This allows the results from the predictive model for demand, built on existing stations, to translate to new stations. 

The key assumption is that the bikeshare is not yet at full capacity, and riders in new areas will not decrease the demand in other areas.

## Is this a good assumption?

- Given that the bikeshare is looking to expand, it's a safe bet that they believe the program is not yet at full capacity
- This is verifiable with existing data — examine trip counts in neighboring stations when a new station opens up. 

**Typically, this is a pretty safe assumption.** But I encourage you to use historical data to verify it!

**Two important columns:**

- `totalDocks`: the total number of docks at each station
- `kioskId`: the unique identifier for each station

## Getting trip data for the Indego bike share

- Available by quarter from: https://www.rideindego.com/about/data/
- I've pulled trip data for 2018 and 2019 (through Q3) and combined into a single CSV file (available in the data folder)

The data page also includes the live status of all of the stations in the system.

In [ ]:
# Get the live station status
stations = gpd.read_file("http://www.rideindego.com/stations/json/")

In [ ]:
stations.head()

## Let's plot the stations, colored by the number of docks

In [ ]:
import contextily as ctx

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

# stations
stations_3857 = stations.to_crs(epsg=3857)
stations_3857.plot(ax=ax, column='totalDocks', legend=True)

# plot the basemap underneath
ctx.add_basemap(ax=ax, crs=stations_3857.crs, url=ctx.providers.CartoDB.DarkMatter)

ax.set_axis_off()

## Load all trips from 2018 and 2019

In [4]:
all_trips = pd.read_csv("./data/indego-trips-2018-2019.csv.tar.gz")

/Users/nhand/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
all_trips.head()

## Dependent variable: total trips by starting station

In [ ]:
start_trips = all_trips.groupby("start_station").size().reset_index(name="total_start_trips")

In [ ]:
# Now merge in the geometry for each station
bike_data = (
    stations[["geometry", "kioskId"]]
    .merge(start_trips, left_on="kioskId", right_on="start_station")
    .to_crs(epsg=3857)
)

Let's plot it...

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

# stations
bike_data.plot(ax=ax, column='total_start_trips', legend=True)

# plot the basemap underneath
ctx.add_basemap(ax=ax, crs=bike_data.crs, url=ctx.providers.CartoDB.DarkMatter)

ax.set_axis_off()

Trips are clearly concentrated in Center City...

## What features to use?

There are lots of possible options. Generally speaking, the possible features fall into a few different categories:

- **Internal**
    - e.g., the number of docks per station
- **Demographic (Census)**    
    - e.g., population, income, percent commuting by car, percent with bachelor's degree or higher, etc
- **Amenities/Disamenities**
    - e.g., distance to nearest crimes, restaurants, parks, within Center City Business District, etc
- **Transportation network**
    - e.g., distance to nearest bus stop, interesection nodes, nearest subway stop
- **Neighboring stations**
    - e.g., average trips of nearest stations, distance to nearest stations
    
**Let's add a few from each category...**

## 1. Internal characteristics

Let's use the number of docks per stations:

In [ ]:
bike_data = bike_data.merge(stations[["kioskId", "totalDocks"]], on="kioskId")
bike_data.head()

## 2. Census demographic data

We'll try out percent commuting by car first.

In [ ]:
from census import Census

In [ ]:
# Note that using an API key of "None" should work for most use cases
c = Census(key=None)

In [ ]:
# Variable names chosen from: https://api.census.gov/data/2017/acs/acs5/variables.html
# This is the total number of commuters and those commuting by car
variables = ("NAME", "B08134_001E", "B08134_011E")
census_data = pd.DataFrame(c.acs5.state_county_tract(variables, "42", "101", "*"))

# The percent commuting by car
census_data["percent_car"] = census_data["B08134_011E"] / census_data["B08134_001E"]

In [ ]:
census_data.head()

Merge with census tract geometries for Philadelphia:

In [ ]:
import us

In [ ]:
pa_tracts = gpd.read_file(us.states.PA.shapefile_urls('tract'))

In [ ]:
census_data = pa_tracts.merge(
    census_data,
    left_on=["STATEFP10", "COUNTYFP10", "TRACTCE10"],
    right_on=["state", "county", "tract"],
)


Finally, let's merge the census data into our dataframe of features by spatially joining the stations and the census tracts:

In [ ]:
bike_data = gpd.sjoin(
    bike_data,
    census_data.to_crs(bike_data.crs)[["geometry", "percent_car"]],
    op="within",
).drop(labels=['index_right'], axis=1)

In [ ]:
bike_data.head()

"Impute" missing values with the median value:

In [ ]:
missing = bike_data['percent_car'].isnull()
bike_data.loc[missing, 'percent_car'] = bike_data['percent_car'].median()

## Amenities/disamenities

Let's add two new features:

1. Distances to the nearest 10 restaurants from Open Street Map
1. Whether the station is within the Center City Business District

### Restaurants

Search https://wiki.openstreetmap.org/wiki/Map_Features for OSM identifier of restaurants

In [ ]:
restaurants = osm2gpd.get(lng_min, lat_min, lng_max, lat_max, where="amenity=restaurant").to_crs(epsg=3857)
restaurants.head()

Get x/y values for the stations:

In [ ]:
stationsXY = get_xy_from_geometry(bike_data)

In [ ]:
# STEP 1: x/y coordinates of restaurants (in EPGS=3857)
restsXY = get_xy_from_geometry(restaurants.to_crs(epsg=3857))

# STEP 2: Initialize the algorithm
nbrs = NearestNeighbors(n_neighbors=5)

# STEP 3: Fit the algorithm on the "neighbors" dataset
nbrs.fit(restsXY)

# STEP 4: Get distances for stations to neighbors
restsDists, restsIndices = nbrs.kneighbors(stationsXY)

# STEP 5: add back to the original dataset
bike_data['logDistRests'] = np.log10(restsDists.mean(axis=1))

In [ ]:
bike_data.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

# stations
bike_data.plot(ax=ax, column='logDistRests', legend=True)

# plot the basemap underneath
ctx.add_basemap(ax=ax, crs=bike_data.crs, url=ctx.providers.CartoDB.DarkMatter)

ax.set_axis_off()

### Center City Business District

Available from [OpenDataPhilly](https://www.opendataphilly.org/dataset/center-city-business-improvement-district)

In [ ]:
url = "http://data.phl.opendata.arcgis.com/datasets/95366b115d93443eae4cc6f498cb3ca3_0.geojson"
cc_bid = gpd.read_file(url).to_crs(epsg=3857)

In [ ]:
cc_bid

In [ ]:
cc_bid_geo = cc_bid.iloc[0].geometry

cc_bid_geo

In [ ]:
bike_data['within_cc_bid'] = bike_data.geometry.within(cc_bid_geo).astype(int)

In [ ]:
bike_data.head()

### Transportation Network

- Let's add a feature that calculates the distance to the nearest intersections
- We can use the osmnx package

In [ ]:
import osmnx as ox

In [ ]:
xmin, ymin, xmax, ymax = bike_data.to_crs(epsg=4326).total_bounds

In [ ]:
ox.graph_from_bbox?

In [ ]:
G = ox.graph_from_bbox(ymax, ymin, xmax, xmin)

In [ ]:
G

In [ ]:
interesections = ox.graph_to_gdfs(G, nodes=True, edges=False).to_crs(epsg=3857)

In [ ]:
interesections

In [ ]:
# STEP 1: x/y coordinates of restaurants (in EPGS=3857)
intersectionsXY = get_xy_from_geometry(interesections.to_crs(epsg=3857))

# STEP 2: Initialize the algorithm
nbrs = NearestNeighbors(n_neighbors=10)

# STEP 3: Fit the algorithm on the "neighbors" dataset
nbrs.fit(intersectionsXY)

# STEP 4: Get distances for stations to neighbors
interDists, interIndices = nbrs.kneighbors(stationsXY)

# STEP 5: add back to the original dataset
bike_data['logIntersectionDists'] = np.log10(interDists.mean(axis=1))

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

# stations
bike_data.plot(ax=ax, column='logIntersectionDists', legend=True)

# plot the basemap underneath
ctx.add_basemap(ax=ax, crs=bike_data.crs, url=ctx.providers.CartoDB.DarkMatter)

ax.set_axis_off()

### Neighboring Stations

- We need to include features that encodes the fact that demand for a specific station is likely related to the demand in neighboring stations
- This idea is known as **spatial lag**

We will add two new features:

1. The average distance to the nearest 5 stations
1. The average trip total for the nearest 5 stations

First, find the nearest 5 stations:

In [ ]:
k = 6
nbrs = NearestNeighbors(n_neighbors=k)
nbrs.fit(stationsXY)

stationDists, stationIndices = nbrs.kneighbors(stationsXY)

**Notes**

- We are matching the stations to themselves to find the nearest neighbors
- The closest match will always be the same station (distance of 0)
- So we fit for $k+1$ neighbors and will remove the closest neighbor

In [ ]:
bike_data['logStationDists'] = np.log10(stationDists[:,1:].mean(axis=1))

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

# stations
bike_data.plot(ax=ax, column='logStationDists', legend=True)

# plot the basemap underneath
ctx.add_basemap(ax=ax, crs=bike_data.crs, url=ctx.providers.CartoDB.DarkMatter)

ax.set_axis_off()

In [ ]:
# the total trips for the stations
total_start_trips = bike_data['total_start_trips'].values

# get the trips for the 5 nearest neighbors (ignoring first match)
neighboring_trips = total_start_trips[stationIndices[:,1:]]

# add to features
bike_data['laggedTrips'] = neighboring_trips.mean(axis=1)

## Let's fit a model!

In [ ]:
bike_data.head()

## Perform our test/train split

We'll use a 60%/40% split

In [ ]:
# Remove unnecessary columns
bike_features = bike_data.drop(labels=["geometry", "kioskId", "start_station"], axis=1)

In [ ]:
# Split the data 60/40
train_set, test_set = train_test_split(
    bike_features,
    test_size=0.4,
    random_state=42,
)

# the target labels
y_train = np.log(train_set["total_start_trips"])
y_test = np.log(test_set["total_start_trips"])

In [ ]:
train_set = train_set.drop(labels=['total_start_trips'], axis=1)
test_set = test_set.drop(labels=['total_start_trips'], axis=1)

## Random forest results

Let's run a simple grid search to try to optimize our hyperparameters.

In [ ]:
regr = make_pipeline(
    StandardScaler(), RandomForestRegressor(random_state=42)
)

In [ ]:
model_name = "randomforestregressor"
param_grid = {
    f"{model_name}__n_estimators": [5, 10, 15, 20, 30, 50, 100, 200],
    f"{model_name}__max_depth": [2, 5, 7, 9, 13, 21, 33, 51, 100],
}

param_grid

In [ ]:
# Create the grid and use 3-fold CV
grid = GridSearchCV(regr, param_grid, cv=3)

# Run the search
grid.fit(train_set.values, y_train);

In [ ]:
# Evaluate the best random forest model
best_random = grid.best_estimator_
best_random.score(test_set, y_test)

In [ ]:
linear = make_pipeline(StandardScaler(), LinearRegression())

# Fit on train set
linear.fit(train_set, y_train)

# Evaluate on test set
linear.score(test_set, y_test)

## Feature importances

In [ ]:
# The regressor
regressor = grid.best_estimator_["randomforestregressor"]

# Create the dataframe with importances
importance = pd.DataFrame(
    {"Feature": train_set.columns, "Importance": regressor.feature_importances_}
)

# Sort importance in descending order and get the top
importance = importance.sort_values("Importance", ascending=False).iloc[:50]

# Plot
importance.hvplot.barh(x="Feature", y="Importance", flip_yaxis=True, height=700)

In [ ]:
importance

## Let's analyze the spatial structure of the predictions visually

We'll plot the predicted and actual trip values

In [ ]:
# Extract the test data from the original dataset
# This will include the geometry data
X = bike_data.loc[test_set.index]

In [ ]:
# Convert the predicted test values from log
X['prediction'] = np.exp(grid.best_estimator_.predict(test_set))

In [ ]:
# Plot two columns
fig, axs = plt.subplots(ncols=2, figsize=(10,10))

# Predicted values
X.plot(ax=axs[0], column='prediction')
ctx.add_basemap(ax=axs[0], crs=X.crs, url=ctx.providers.CartoDB.DarkMatter)
axs[0].set_title("Predicted Trip Counts")

# Actual values
X.plot(ax=axs[1], column='total_start_trips')
ctx.add_basemap(ax=axs[1], crs=X.crs, url=ctx.providers.CartoDB.DarkMatter)
axs[1].set_title("Actual Trip Counts")


axs[0].set_axis_off()
axs[1].set_axis_off()

## Exercise: can we improve the model?

Yes! This is a classic example of underfitting. With only ten features, we should be able to improve with new features?

**Options**
- Additional census demographic data: 
    - e.g., population, income, percent with bachelor's degree or higher
    - See https://api.census.gov/data/2017/acs/acs5/variables.html for column names!
- Amenities / disamenities
    - Lots of options from OpenDataPhilly and OpenStreetMap
- Transportation network
    - Distance to the nearest bus station is a good place to start (see https://wiki.openstreetmap.org/wiki/Map_Features for the amenity tag)
- Changing $k$ values for distance-based features
    - Experiment with different values of $k$ to see if they improve the model

## Other options

When trying to improve the accuracy of the model, another option is incorporating additional data. In this case, we can look to other cities and include trip data for these cities. Some good places to start:

- [Boston](https://www.bluebikes.com/)
- [Chicago](https://www.divvybikes.com/)
- [Washington D.C](https://www.capitalbikeshare.com/)